# Simplifying the column names (CSV header)

The CSV types and their columns have a Brazilian Portuguese description in
http://docs.scielo.org/projects/scielo-processing/pt/latest/public_reports.html.

However, the column names have some issues:

* Sometimes they're way too long;
* Almost always they have some whitespace or other non-alphanumeric character;
* Some names have trailing whitespaces;
* There are multiple languages in the `journals_kbart.csv`,
  following the `Brazilian Portuguese Name (english_name)` format;
* They might include redundant/ambiguous/misleading parts.

In summary, they're difficult to deal with
when we're performing some exploratory data analysis
or otherwise using them in the middle of some source code,
in almost any language.
Our goal is to simplify that to keep it similar to a `snake_case` format.

In [1]:
import csv
from glob import glob

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 200 # Default is 60

## Current rows

These are all the titles from every CSV file:

In [3]:
for fname in glob("tabs_network/*.csv"):
    with open(fname) as f:
        cr = csv.reader(f)
        print(next(cr))

['extraction date', 'study unit', 'collection', 'ISSN SciELO', "ISSN's", 'title at SciELO', 'title thematic areas', 'title is agricultural sciences', 'title is applied social sciences', 'title is biological sciences', 'title is engineering', 'title is exact and earth sciences', 'title is health sciences', 'title is human sciences', 'title is linguistics, letters and arts', 'title is multidisciplinary', 'title current status', 'document publishing ID (PID SciELO)', 'document publishing year', 'document is citable', 'document type', 'document languages', 'document pt', 'document es', 'document en', 'document other languages']
['extraction date', 'study unit', 'collection', 'ISSN SciELO', "ISSN's", 'title at SciELO', 'title thematic areas', 'title is agricultural sciences', 'title is applied social sciences', 'title is biological sciences', 'title is engineering', 'title is exact and earth sciences', 'title is health sciences', 'title is human sciences', 'title is linguistics, letters and

The full set of names is:

In [4]:
names = set()
for fname in glob("tabs_network/*.csv"):
    with open(fname) as f:
        cr = csv.reader(f)
        names.update(next(cr))
np.array(sorted(names))

array(['+6 authors', '0 authors', '1 author', '2 authors', '3 authors',
       '4 authors', '5 authors',
       'Data do primeiro fascículo (date_first_issue_online)',
       'Data do último fascículo publicado (date_last_issue_online)',
       'ID de publicação pai (parent_publication_title_id)',
       'ID de publicação prévia (preceding_publication_title_id)',
       'ID do periódico no SciELO (title_id)', 'ISSN SciELO',
       'ISSN impresso (print_identifier)',
       'ISSN online (online_identifier)', "ISSN's",
       'Título do Periódico (publication_title)', 'accesses to abstract',
       'accesses to epdf', 'accesses to html', 'accesses to pdf',
       'accesses year', 'alpha frequency', 'altmetrics url', 'authors',
       'citable documents', 'citable documents at 2013',
       'citable documents at 2014', 'citable documents at 2015',
       'citable documents at 2016', 'citable documents at 2017',
       'citable documents at 2018', 'cobertura (coverage_depth)',
       'coll

Sometimes the columns names have some misleading stuff we can fix, like:

- Extra trailing whitespace
- Distinct/mixed letter cases
- Redundant parentheses structure like
  `Plain text column description in Portuguese (snake_case_descr_in_english)`
- Symbols like `'` and `+`

We can fix that by keeping only the parenthesized code,
removing some less meaningful common words,
shortening some lenghty words, and
performing some replacements:

In [5]:
def normalize_column_title(name):
    import re
    name_unbracketed = re.sub(r".*\((.*)\)", r"\1",
                              name.replace("(in months)", "in_months"))
    words = re.sub("[^a-z0-9+_ ]", "", name_unbracketed.lower()).split()
    ignored_words = ("at", "the", "of", "and", "google", "scholar", "+")
    replacements = {
        "document": "doc",
        "documents": "docs",
        "frequency": "freq",
        "language": "lang",
    }
    return "_".join(replacements.get(word, word)
                    for word in words if word not in ignored_words) \
              .replace("title_is", "is")

With Pandas, its use should be straightforward.

In [6]:
network_journals = pd.read_csv("tabs_network/journals.csv").rename(columns=normalize_column_title)
network_journals.columns

Index(['extraction_date', 'study_unit', 'collection', 'issn_scielo', 'issns',
       'title_scielo', 'title_thematic_areas', 'is_agricultural_sciences',
       'is_applied_social_sciences', 'is_biological_sciences',
       'is_engineering', 'is_exact_earth_sciences', 'is_health_sciences',
       'is_human_sciences', 'is_linguistics_letters_arts',
       'is_multidisciplinary', 'title_current_status', 'title_subtitle_scielo',
       'short_title_scielo', 'short_iso', 'title_pubmed', 'publisher_name',
       'use_license', 'alpha_freq', 'numeric_freq_in_months',
       'inclusion_year_scielo', 'stopping_year_scielo', 'stopping_reason',
       'date_first_doc', 'volume_first_doc', 'issue_first_doc',
       'date_last_doc', 'volume_last_doc', 'issue_last_doc', 'total_issues',
       'issues_2018', 'issues_2017', 'issues_2016', 'issues_2015',
       'issues_2014', 'issues_2013', 'total_regular_issues',
       'regular_issues_2018', 'regular_issues_2017', 'regular_issues_2016',
       'regul